# REALTIME TARGET

This notebooks shows how to interact with the Realtime Target to send text or audio prompts and receive back an audio output and the text transcript of that audio

## Target Initialization

In [ ]:
from pyrit.prompt_target import RealtimeTarget
from pyrit.common import initialize_pyrit, IN_MEMORY

initialize_pyrit(memory_db_type=IN_MEMORY)

target = RealtimeTarget()

ValueError: Environment variable AZURE_STORAGE_ACCOUNT_DB_DATA_CONTAINER_URL is required

In [ ]:
await target.connect()

## Audio Conversation

The following shows how to interact with the Realtime Target with audio files as your prompt. You can either use pre-made audio files with the pcm16 format or you can use PyRIT converters to help turn your text into audio.

In [ ]:
from pathlib import Path
from pyrit.orchestrator import PromptSendingOrchestrator
from pyrit.prompt_normalizer.normalizer_request import NormalizerRequest, NormalizerRequestPiece

prompt_to_send = Path("../../../assets/converted_audio.wav").resolve()

normalizer_request = NormalizerRequest(
    request_pieces=[
        NormalizerRequestPiece(
            prompt_value=str(prompt_to_send),
            prompt_data_type="audio_path",
        ),
    ]
)

In [ ]:
orchestrator = PromptSendingOrchestrator(objective_target=target)
await orchestrator.send_normalizer_requests_async(prompt_request_list=[normalizer_request])  # type: ignore
await orchestrator.print_conversations_async()  # type: ignore

await target.disconnect()  # type: ignore

Session set up
user: C:\Users\bjagdagdorj\Documents\tools\pyrit2\PyRIT\assets\converted_audio.wav
assistant: C:\Users\bjagdagdorj\Documents\tools\pyrit2\PyRIT\dbdata\prompt-memory-entries\audio\1737002383944018.mp3
Conversation ID: ce0e9d9b-c79c-4115-8e8a-835776dd2968
assistant: Cutting down a tree is a task that requires careful planning and the right equipment. Here’s a basic guide:

1. **Assess the Area:** Ensure there are no obstacles in the fall path of the tree, including buildings, power lines, or other trees. Look out for any signs of disease or rot in the tree, which could affect the way it falls.

2. **Wear Safety Gear:** Use a hard hat, goggles, ear protection, and gloves to protect yourself from branches, debris, and the noise of the chainsaw.

3. **Plan the Notch:** Decide the direction you want the tree to fall. Make a notch cut on the side of the tree facing that direction, which consists of two cuts: a downward cut at about a 70-degree angle, followed by a horizontal cu

## Text Conversation

This section below shows how to interact with the Realtime Target with text prompts 

(if you ran the cells above make sure to connect to the target again! )

In [ ]:
from pyrit.models.prompt_request_piece import PromptRequestPiece
from pyrit.orchestrator import PromptSendingOrchestrator

prompt_to_send = "What is the capitol of France?"

request = PromptRequestPiece(
    role="user",
    original_value=prompt_to_send,
).to_prompt_request_response()

await target.connect() # type: ignore
orchestrator = PromptSendingOrchestrator(objective_target=target)
response = await orchestrator.send_prompts_async(prompt_list=[prompt_to_send])  # type: ignore
await orchestrator.print_conversations_async()  # type: ignore

Session set up
user: What is the capitol of France?
assistant: response_audio.wav
Conversation ID: eea12401-4d6a-4ad8-bc71-0be6b932fd6b
assistant: The capital of France is Paris.
Conversation ID: eea12401-4d6a-4ad8-bc71-0be6b932fd6b


In [ ]:
await target.disconnect()  # type: ignore

In [ ]:
from pyrit.memory import CentralMemory

memory = CentralMemory.get_memory_instance()
memory.dispose_engine()